In [1]:
# 2->3->2_1 (y = x0 + x1)
import keras
import numpy as np

from keras.layers import Dense, Input
from keras.models import Model

x_train = np.array([[0,1],[2,3],[4,5]])
# y = x0 + x1
# y_train = np.sum(x_train, axis=1)
y_train = np.array([[x.sum()] for x in x_train])   # 2차원 array 형태
print('x_train:\n', x_train)
print('y_train:\n', y_train)

x_test = np.array([[6,7],[8,9]])
# y_test = np.sum(x_test, axis=1)
y_test = np.array([[x.sum()] for x in x_test])   # 2차원 array 형태
print('x_test:\n', x_test)
print('y_test:\n', y_test)

d1 = Dense(3, name='d1')   # d1 layer
d2 = Dense(2, name='d2')   # d2 layer
d3 = Dense(1, name='d3')   # d3 layer

x = Input(shape=(2,), name='x')    # input Tensor 'x'
d1_out = d1(x)   # d1 layer가 Tensor x를 입력으로 받아서, d1_out Tensor 출력
d2_out = d2(d1_out)   # Tensor d1_out를 입력으로 받아서, d2_out Tensor 출력
d3_out = d3(d2_out)   # Tensor d2_out를 입력으로 받아서, d3_out Tensor 출력

model = Model(x, d3_out)   # Model(inputs=x, outputs=d3_out)
model.summary()

Using TensorFlow backend.


x_train:
 [[0 1]
 [2 3]
 [4 5]]
y_train:
 [[1]
 [5]
 [9]]
x_test:
 [[6 7]
 [8 9]]
y_test:
 [[13]
 [17]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (InputLayer)               (None, 2)                 0         
_________________________________________________________________
d1 (Dense)                   (None, 3)                 9         
_________________________________________________________________
d2 (Dense)                   (None, 2)                 8         
_________________________________________________________________
d3 (Dense)                   (None, 1)                 3         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


#### layer별 weights & bias 확인하기

In [2]:
d1_weights = d1.get_weights()
print('d1_weights type:', type(d1_weights)) # list
print('d1_weights:'); print(d1_weights)  # weight와 bias 2개의 numpy array

d1 = model.get_layer('d1')
d1_weights = d1.get_weights()
print('d1_weights type:', type(d1_weights)) # list
print('d1_weights:');
print(d1_weights[0]) # d1 layer의 input weights
print(d1_weights[1]) # d1 layer의 input bias

# d2_weights = d2.get_weights()
# d3_weights = d3.get_weights()
# print('d2_weights:'); print(d2_weights)
# print('d3_weights:'); print(d3_weights)

d1_weights type: <class 'list'>
d1_weights:
[array([[-0.34320205,  0.557724  ,  0.69108224],
       [-0.6989583 , -0.9579384 ,  0.98622274]], dtype=float32), array([0., 0., 0.], dtype=float32)]
d1_weights type: <class 'list'>
d1_weights:
[[-0.34320205  0.557724    0.69108224]
 [-0.6989583  -0.9579384   0.98622274]]
[0. 0. 0.]


#### weights를 원하는 값으로 변경하기

In [3]:
# weight 값 변경하기(원하는 값으로...) : pre-trained model의 weight 활용 가능
weights = [np.array([[1,3,5], [2,4,6]], dtype=np.float32), 
           np.array([1,2,3], dtype=np.float32)]  # 2개의 numpy array(weghts와 bias)
weights

[array([[1., 3., 5.],
        [2., 4., 6.]], dtype=float32), array([1., 2., 3.], dtype=float32)]

In [4]:
d1 = model.get_layer('d1')
d1.set_weights(weights)   # weights 값 변경

# 변경된 weights 값 확인
d1 = model.get_layer('d1')
d1_weights = d1.get_weights()
print('d1_weights:')
print(d1_weights[0]) # d1 layer의 input weights
print(d1_weights[1]) # d1 layer의 input bias

d1_weights:
[[1. 3. 5.]
 [2. 4. 6.]]
[1. 2. 3.]


In [5]:
# dense2 layer weight 값 변경하기
weights2 = [np.array([[1,4], [2,5], [3,6]], dtype=np.float32), 
            np.array([1,2], dtype=np.float32)]  # 2개의 numpy array(weghts와 bias)
d2 = model.get_layer('d2')
d2.set_weights(weights2)   # weights 값 변경

# 변경된 weights 값 확인
d2 = model.get_layer('d2')
d2_weights = d2.get_weights()
print('d2_weights:')
print(d2_weights[0]) # d1 layer의 input weights
print(d2_weights[1]) # d1 layer의 input bias

d2_weights:
[[1. 4.]
 [2. 5.]
 [3. 6.]]
[1. 2.]


In [6]:
# dense3 layer weight 값 변경하기
weights3 = [np.array([[1], [2]], dtype=np.float32), 
            np.array([1], dtype=np.float32)]  # 2개의 numpy array(weghts와 bias)
d3 = model.get_layer('d3')
d3.set_weights(weights3)   # weights 값 변경

# 변경된 weights 값 확인
d3 = model.get_layer('d3')
d3_weights = d3.get_weights()
print('d3_weights:')
print(d3_weights[0]) # d1 layer의 input weights
print(d3_weights[1]) # d1 layer의 input bias

d3_weights:
[[1.]
 [2.]]
[1.]


#### get_weights를 이용해서 forward propergation 함수 만들기

In [7]:
def computer_layer_output(layer, input_batch):
    weights = layer.get_weights()
    return np.matmul(input_batch, weights[0]) + weights[1]

In [8]:
# keras dense layer predict와 computer_layer_output predict가 같은지 확인
model_d1 = Model(model.input, d1.output)
model_d1.summary()
d1_predict = model_d1.predict(x_test)
print('d1_predict'); print(d1_predict)

d1_out = computer_layer_output(d1, x_test)
print('computer_layer_output'); print(d1_out)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (InputLayer)               (None, 2)                 0         
_________________________________________________________________
d1 (Dense)                   (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
d1_predict
[[21. 48. 75.]
 [27. 62. 97.]]
computer_layer_output
[[21. 48. 75.]
 [27. 62. 97.]]


In [9]:
# computer_layer_output 함수로 전체 laryer 재구성 하기
d1_out = computer_layer_output(d1, x_test)
d2_out = computer_layer_output(d2, d1_out)
d3_out = computer_layer_output(d3, d2_out)

print('d3_out'); print(d3_out)
print('model predict'); print(model.predict(x_test))

d3_out
[[1896.]
 [2448.]]
model predict
[[1896.]
 [2448.]]
